In [150]:
import pandas as pd
import yfinance as yf
import sqlalchemy
import streamlit as st
import yfinance as yf
import datetime as dt
import os
import time
import pandas_ta as ta
from backtesting import Backtest, Strategy
import numpy as np
from backtesting.lib import resample_apply
from backtesting.lib import crossover, TrailingStrategy
from pandas_datareader import data
from backtesting.test import SMA

In [151]:
engine=sqlalchemy.create_engine('sqlite:///günlük.db')
enginew=sqlalchemy.create_engine('sqlite:///haftalik.db')

In [152]:
index=1
bsymbols1=pd.read_csv('hepsi.csv',header=None)
bsymbols=bsymbols1.iloc[:,0].to_list()
bnameslist = bsymbols1.iloc[:,1].to_list()
for bticker, bnames in zip (bsymbols[:5],bnameslist[:5]):
    print(index,bticker,end="\r")
    st.write(f"⏳ {index,bticker} downloaded")
    index += 1
    df=yf.download(bticker,period="1y",interval='1d',auto_adjust=True )
    ohlcv_dict = {'Open': 'first',
      'High': 'max',
      'Low': 'min',
      'Close': 'last',
      'Volume': 'sum'
     }
    df.to_sql(bnames,engine, if_exists='replace')
    df2w = df.resample('W-FRI').agg(ohlcv_dict)
    df2w.dropna(inplace=True)
    df2w.to_sql(bnames,enginew, if_exists='replace')

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [153]:
symbols= pd.read_sql('SELECT name FROM sqlite_master WHERE type="table"',engine)
symbols

,name
0,Gold Feb 23
1,Silver Mar 23
2,Brent Crude Oil Last Day Financ
3,"Chicago SRW Wheat Futures,Mar-2"
4,Natural Gas Jan 23


In [160]:
df

,Open,High,Low,Close,Volume
Date,,,,,
2022-01-10,4.144,4.213,3.977,4.079,177628
2022-01-11,4.119,4.261,4.007,4.249,121623
2022-01-12,4.225,4.879,4.186,4.857,217499
2022-01-13,4.791,4.844,4.245,4.270,221612
2022-01-14,4.263,4.305,4.097,4.262,108856
...,...,...,...,...,...
2023-01-04,4.008,4.219,3.900,4.172,99759
2023-01-05,4.155,4.175,3.651,3.720,116682
2023-01-06,3.764,3.839,3.520,3.710,105050


In [168]:
class SmaCross(Strategy):
    n1 = 10
    n2 = 20

    def init(self):
        close = self.data.Close
        self.sma1 = self.I(SMA, close, self.n1)
        self.sma2 = self.I(SMA, close, self.n2)

    def next(self):
        if crossover(self.sma1, self.sma2):
            self.buy()
        elif crossover(self.sma2, self.sma1):
            self.sell()
bt = Backtest(df, SmaCross,
              cash=10000, commission=.002,
              exclusive_orders=True)

output= bt.run()
output
#bt.plot()

Start                     2022-01-10 00:00:00
End                       2023-01-10 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                    89.72332
Equity Final [$]                 19137.297095
Equity Peak [$]                  19891.819225
Return [%]                          91.372971
Buy & Hold Return [%]              -12.208872
Return (Ann.) [%]                   90.882646
Volatility (Ann.) [%]              158.347275
Sharpe Ratio                         0.573945
Sortino Ratio                        1.867258
Calmar Ratio                         2.221198
Max. Drawdown [%]                  -40.916054
Avg. Drawdown [%]                   -12.04293
Max. Drawdown Duration      194 days 00:00:00
Avg. Drawdown Duration       29 days 00:00:00
# Trades                                    9
Win Rate [%]                        55.555556
Best Trade [%]                      63.585874
Worst Trade [%]                    -16.413704
Avg. Trade [%]                    

In [169]:
def BT():
    returns=[]
    for symbol in symbols.name:
        qry=f"""SELECT*FROM '{symbol}'"""
        
        df10=pd.read_sql(qry,engine).set_index('Date')
        df10.index=pd.to_datetime(df10.index)
        bt=Backtest(df10,SmaCross, cash=5000, commission=.002)
        output=bt.run()
        returns.append(output['Return [%]'])
    frame=pd.DataFrame(returns,index=symbols.name,columns=['ret'])
    top_5=frame.nlargest(5,'ret')
    return top_5

In [170]:
BT()

,ret
name,
Brent Crude Oil Last Day Financ,37.815440
"Chicago SRW Wheat Futures,Mar-2",4.402920
Gold Feb 23,-0.025809
Silver Mar 23,-1.085481
Natural Gas Jan 23,-100.000000


In [171]:
returns=[]
for symbol in symbols.name:
    qry=f"""SELECT*FROM '{symbol}'"""
    
    df10=pd.read_sql(qry,engine).set_index('Date')
    df10.index=pd.to_datetime(df10.index)
    bt=Backtest(df10,SmaCross, cash=10000, commission=.002)
    output=bt.run()
    print(symbol)
    print(output)
    returns.append(output['Return [%]'])
    #frame=pd.DataFrame(returns,index=symbols.name,columns=['ret'])
    #top_5=frame.nlargest(5,'ret')


Gold Feb 23
Start                     2022-01-10 00:00:00
End                       2023-01-10 00:00:00
Duration                    365 days 00:00:00
Exposure Time [%]                   90.079365
Equity Final [$]                  9996.773878
Equity Peak [$]                     10818.774
Return [%]                          -0.032261
Buy & Hold Return [%]                4.637457
Return (Ann.) [%]                   -0.032261
Volatility (Ann.) [%]               14.064496
Sharpe Ratio                              0.0
Sortino Ratio                             0.0
Calmar Ratio                              0.0
Max. Drawdown [%]                  -19.262808
Avg. Drawdown [%]                     -4.6737
Max. Drawdown Duration      308 days 00:00:00
Avg. Drawdown Duration       65 days 00:00:00
# Trades                                    1
Win Rate [%]                              0.0
Best Trade [%]                      -0.034387
Worst Trade [%]                     -0.034387
Avg. Trade [%]        